In [1]:
import gensim
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.tokenize import sent_tokenize
import nltk.data
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


False

# Prepare the inputs

In [ ]:
# The training data can be downloaded from 
# https://www.kaggle.com/c/word2vec-nlp-tutorial/data
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
sentences = []
for review in unlabeled_train["review"]:
    # Split the paragraph into sentences
    try:
        review = review.decode('ascii', 'ignore')
        raw_sentences = sent_tokenize(review.strip())
        for rs in raw_sentences:
            # If the sentence is nonempty convert to a list of words
            if len(rs) > 0:
                # Remove HTML tags and nonletters, then lowercase
                review_text = BeautifulSoup(review, "html.parser").get_text()
                review_text = re.sub("[^a-zA-Z]"," ", review_text)
                review_text = review_text.lower()

                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append(review_text.split())
    except Exception as e:
        print review
        print e

# Test
print sentences[:1]

# Should be:
# [[u'watching', u'time', u'chasers', u'it', u'obvious', u'that', u'it', u'was', u'made', u'by', u'a', u'bunch', u'of', u'friends', u'maybe', u'they', u'were', u'sitting', u'around', u'one', u'day', u'in', u'film', u'school', u'and', u'said', u'hey', u'let', u's', u'pool', u'our', u'money', u'together', u'and', u'make', u'a', u'really', u'bad', u'movie', u'or', u'something', u'like', u'that', u'what', u'ever', u'they', u'said', u'they', u'still', u'ended', u'up', u'making', u'a', u'really', u'bad', u'movie', u'dull', u'story', u'bad', u'script', u'lame', u'acting', u'poor', u'cinematography', u'bottom', u'of', u'the', u'barrel', u'stock', u'music', u'etc', u'all', u'corners', u'were', u'cut', u'except', u'the', u'one', u'that', u'would', u'have', u'prevented', u'this', u'film', u's', u'release', u'life', u's', u'like', u'that']]

# Train the Model

In [ ]:
# Set values for various parameters
sg=1                  # Use skip_gram as opposed to cbow
num_features = 300    # Word vector dimensionality                      
min_word_count = 20   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [ ]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sg=sg, sample = downsampling)
model.save('./mymodel_movie_reviews')
print "...Done"

# Use the model

In [2]:
new_model = gensim.models.Word2Vec.load('./mymodel_movie_reviews')

In [4]:
new_model.most_similar(positive=['husband', 'man'], negative='wife', topn=1)

[(u'woman', 0.35663580894470215)]

In [5]:
new_model.doesnt_match("scene costume tree actor".split())
# Should be 'tree'

'tree'

In [3]:
new_model.similarity('good', 'bad')

0.48511547857168397

In [6]:
# Print the distributed representation of a word
plot_vec = new_model['plot']
print len(plot_vec)
print plot_vec

300
[  1.29108980e-01  -2.41659917e-02  -3.53948474e-02   6.83904961e-02
  -1.00694224e-03   4.18567844e-03   7.70894215e-02  -1.00809708e-01
  -2.12428104e-02  -2.05542948e-02   1.65275574e-01  -1.78873584e-01
   5.13368323e-02   2.64769793e-01   4.11492325e-02   1.98992908e-01
   3.51657793e-02   9.09533910e-03  -1.96469054e-01  -8.49656686e-02
  -7.29236975e-02   9.03672576e-02  -1.82572696e-02   1.43599316e-01
   9.71580371e-02   7.90011212e-02   1.28936529e-01  -3.87321860e-02
   2.65798513e-02   8.90215933e-02  -6.58069551e-02  -9.62048620e-02
   4.84529734e-02  -1.92115352e-01  -1.43481731e-01  -4.66957763e-02
  -1.03409141e-01   6.42854795e-02   3.00124139e-02   1.15883842e-01
  -7.01519176e-02   6.06129654e-02  -6.15574569e-02  -1.14056312e-01
   8.58952478e-02  -1.19828664e-01   5.71376123e-02  -1.48606867e-01
  -1.06906697e-01  -1.95837580e-02  -1.52783364e-01  -1.58696249e-02
   8.73960704e-02  -4.94958945e-02  -4.38045012e-03  -5.69973737e-02
   1.76323265e-01  -4.54487242